In [1]:
import re
import pickle
import time

import numpy as np
import spacy

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from sklearn.svm import SVC

import session_info

In [2]:
session_info.show()

In [3]:
# Загружаем модель из spacy
nlp = spacy.load('ru_core_news_sm')

In [4]:
# Функция для удаления пунктуации и чисел.
def text_process(text):
    text = text.lower()
    # удаляем пунктуацию
    text = re.sub(r'[^\w0-9\s]', '', text)
    # удаляем числа
    text = re.sub(r'\d+', '', text)
    
    return text

In [5]:
# Удаляем частице "не" из списка стоп-слов.
deselect_stop_words = ['не']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


# Процедура удаления стоп-слов.
def remove_stop_words(text):
    #токенизация текста
    doc = nlp(text)

    clean_text = []
    
    for token in doc:
        if not token.is_stop: 
            clean_text.append(token.text)
    return ' '.join(clean_text)


# Функция предобработки текста
def preprocessing(text):
    text_preprocessing = remove_stop_words(text_process(text))
    return text_preprocessing


# Функция для предсказания является заголовок новости фейком или нет.
def predict_fake(news, use, model):
    embedding = use(news)
    embedding = tf.reshape(embedding, (1,-1)).numpy()
    if model.predict(embedding)[0] == 0:
        result = "НЕ ФЕЙК"
    else:
        result = "ФЕЙК"
    return result

In [6]:
# Загружаем модель USE
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [7]:
# Загружаем ранее сохранённый обученный SVC
my_model_svc = pickle.load(open('result_model.pickle','rb'))

In [8]:
# Примеры фейковых и настоящих заголовков.
news_examples = ["Годзилла снова наносит удар по Токио.", "Переоцененный доллар: как ФРС оказалась в тупике", "Конгресс собирается дополнительно выделить пять триллионов долларов на борьбу с Десептиконами.", "Разрушение мифа: отечественный автопром живет и за рубежом.", "Дружелюбный сосед Человек-Паук баллотируется в мэры Нью-Йорка.", "ПИФы телекоммуникаций стали фаворитами недели", "Тони Старк изобрёл вечный двигатель.", "Британская полиция перепутала свадебные декорации с НЛО."]
preprocess_news = [preprocessing(text) for text in news_examples]

In [9]:
start_time = time.time()

for news in preprocess_news:
    print(news,"   ", predict_fake(news, use, my_model_svc))

end_time = time.time()
print()
print("Время обработки ", len(preprocess_news)," заголовков", round(end_time-start_time, 3))
print("Время обработки одного заголовка ", round(end_time-start_time, 3)/len(preprocess_news))

годзилла снова наносит удар токио     НЕ ФЕЙК
переоцененный доллар фрс оказалась тупике     НЕ ФЕЙК
конгресс собирается дополнительно выделить пять триллионов долларов борьбу десептиконами     НЕ ФЕЙК
разрушение мифа отечественный автопром живет рубежом     НЕ ФЕЙК
дружелюбный сосед человекпаук баллотируется мэры ньюйорка     ФЕЙК
пифы телекоммуникаций стали фаворитами недели     НЕ ФЕЙК
тони старк изобрёл вечный двигатель     НЕ ФЕЙК
британская полиция перепутала свадебные декорации нло     НЕ ФЕЙК

Время обработки  8  заголовков 5.511
Время обработки одного заголовка  0.688875


In [10]:
# Видим, что есть к чему стремится.

In [11]:
# Функция позволит вводить строку для обработки с клавиатуры.
def main():
    while True:
        news_head = input("Введите заголовок новости:\n\n").rstrip()
        if news_head == "exit":
            break
                
        start_time = time.time()
        
        preprocessed_news_head = preprocessing(news_head)
        print(news_head," ", predict_fake(preprocessed_news_head, use, my_model_svc))
        print("\n")
        
        end_time = time.time()

        print("Время обработки = ", round(end_time-start_time, 3))
        print()

In [12]:
main()

Введите заголовок новости:

Гарри Поттер выиграл кубок по квиддичу.
Гарри Поттер выиграл кубок по квиддичу.   НЕ ФЕЙК


Время обработки =  0.092

Введите заголовок новости:

Комплексный подход: как облегчить отказ от курения
Комплексный подход: как облегчить отказ от курения   НЕ ФЕЙК


Время обработки =  0.089

Введите заголовок новости:

Лицензии на охоту на вампиров выдавать больше не будут
Лицензии на охоту на вампиров выдавать больше не будут   НЕ ФЕЙК


Время обработки =  0.089

Введите заголовок новости:

Столичная промышленность: какие инструменты поддержки сегодня эффективны
Столичная промышленность: какие инструменты поддержки сегодня эффективны   НЕ ФЕЙК


Время обработки =  0.093

Введите заголовок новости:

exit


In [ ]:
# Да, опредлённо есть к чему стремиться.